In [49]:
import os
import pandas as pd
import dask.dataframe as dd
import numpy as np

from txt_prep import prepareTxt
dirname='../data'

# Importation des `noms de métiers`
# Création du dico des `noms de métiers`
jobNames = pd.read_csv(os.path.join(dirname,'categories_string.csv'))['0'].to_dict()

# Importation du jeu de données d'entrainement
#data=dd.read_json(os.path.join(dirname,'train.json'),lines=False).set_index('Id',n_partitions=10).categorize(columns=['gender'])
data_reduit=pd.read_csv(os.path.join(dirname,'train_set_reduit.csv'),dtype={'description':'string','gender':'category'}).set_index('Id')

train_label=pd.read_csv(os.path.join(dirname, 'train_label.csv'),dtype={'Category':'category'}).set_index('Id')
train_label_reduit=train_label.loc[data_reduit.index.to_list()]

data_reduit['description']=data_reduit.apply(lambda x:prepareTxt(x['description']),axis=1)

In [50]:
print(train_label_reduit.shape)
print(data_reduit.shape)

print(data_reduit.head())
print(train_label_reduit.head())


(30829, 1)
(30829, 2)
                                          description gender
Id                                                          
1   member aicpa wicpa brent graduated university ...      M
30  main interest building scalable software soa d...      M
40  wife julie member since 2001 dwayne also chief...      M
41  work firefighter otrs around madison wisconsin...      M
77  president owner dianne b scinberg associate nu...      F
   Category
Id         
1         9
30       13
40       17
41        4
77       27


In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score 


# When use pipeline class
tfidf_vect=TfidfVectorizer(min_df=1000,ngram_range=(1, 2))
clf=SGDClassifier(loss="modified_huber",early_stopping=True,n_jobs=-1)
pipeline = Pipeline([('vector',tfidf_vect),("clf",clf)])

In [ ]:
# fit
pipeline.fit(df.description,Y)

In [58]:
cross_val_score(pipeline, data_reduit.description,train_label_reduit.Category,scoring="f1_macro", n_jobs=-1)

array([0.22729789, 0.24697638, 0.23205736, 0.24377613, 0.21570618])

In [59]:
#[print(item) for item in tfidf_vect.get_feature_names()]
len(tfidf_vect.get_feature_names())

NotFittedError: Vocabulary not fitted or provided